In [1]:
import pandas as pd

In [21]:
pd. read_csv("data/steam-200k.csv")

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0
...,...,...,...,...,...
199994,128470551,Titan Souls,play,1.5,0
199995,128470551,Grand Theft Auto Vice City,purchase,1.0,0
199996,128470551,Grand Theft Auto Vice City,play,1.5,0
199997,128470551,RUSH,purchase,1.0,0


#### Note: purchase_or_hours always takes the value "1" when the game was purchased and the number of hours played otherwise.

In [25]:
df = pd. read_csv(
    "data/steam-200k.csv",
    header=None,
    names=["userid", "game", "action", "purchase_or_hours", "useless"]
)[["userid", "game", "action", "purchase_or_hours"]]
df.head()

,userid,game,action,purchase_or_hours
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0


### Missing Values

In [31]:
df.isnull().sum()

userid               0
game                 0
action               0
purchase_or_hours    0
dtype: int64

### Appreciation

In this dataset, we do not have an explicit feature translating the rate of appreciation of a game by a user.  Therefore, we consider that the user liked the game if he purchased it or played at least 20 hours.  
God knows there are video games we hated but still played for more than 20 hours, which justifies a higher threshold. However, that would unvoluntarily discard small video games (which take few hours to complete).

Quoique je me dis ptet que c'est mieux :    
- 3 if plays more than 50 hours (even if it's a small game, if it's really good then it can be played multiple times) purchased or not  
- 2 if between 20 and 50 hours, purchased or not (financial reasons not allowing purchase)
- 1 if purchase AND less than 20 hours (likes it but not so much)
- 0 if no purchase AND less than 20 hours (does not like it)

on peut réduire le nombre de classes si problématique

In [56]:
df["score"] = [
    3 if df["purchase_or_hours"][i] >= 50 else
    2 if df["purchase_or_hours"][i] >= 20 and df["purchase_or_hours"][i] < 50 else
    1 if df["purchase_or_hours"][i] < 20 and df["action"][i] == "purchase" else
    0 for i in range(len(df))
]

In [57]:
df

,userid,game,action,purchase_or_hours,score
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,1
1,151603712,The Elder Scrolls V Skyrim,play,273.0,3
2,151603712,Fallout 4,purchase,1.0,1
3,151603712,Fallout 4,play,87.0,3
4,151603712,Spore,purchase,1.0,1
...,...,...,...,...,...
199995,128470551,Titan Souls,play,1.5,0
199996,128470551,Grand Theft Auto Vice City,purchase,1.0,1
199997,128470551,Grand Theft Auto Vice City,play,1.5,0
199998,128470551,RUSH,purchase,1.0,1
